In [1]:
import pandas as pd 
train_lm = pd.read_csv('/home/mila/e/emiliano.penaloza/LLM4REC/data_preprocessed/books/test_tr.csv')
train_te = pd.read_csv('/home/mila/e/emiliano.penaloza/LLM4REC/vae/books/pro_sg_limited/train_te.csv')

In [2]:
s1 = train_lm.sid.unique()
s2 = train_te.sid.unique()


In [3]:

import json
import random 
import os
import pandas as pd
from scipy import sparse
import numpy as np
import argparse
random.seed(2024)

def parse_args():
    parser = argparse.ArgumentParser(description="Data Preprocessing")
    parser.add_argument('--data_name', type=str, default='books',
                        help='name of the dataset')
    return parser.parse_args([])

def get_count(tp, id):
    playcount_groupbyid = tp[[id]].groupby(id, as_index=False)
    count = playcount_groupbyid.size()
    return count



def filter_triplets(tp, min_uc=0, min_sc=0):
    if min_sc > 0:
        itemcount = get_count(tp, 'itemId')
        tp = tp[tp['itemId'].isin(itemcount.index[itemcount['size'] >= min_sc])]
    
    if min_uc > 0:
        usercount = get_count(tp, 'userId')
        tp = tp[tp['userId'].isin(usercount.index[usercount['size'] >= min_uc])]
    
    usercount, itemcount = get_count(tp, 'userId'), get_count(tp, 'itemId')
    return tp, usercount, itemcount



def split_train_test_proportion(data, test_prop=0.2):
    data_grouped_by_user = data.groupby('userId')
    tr_list, te_list = list(), list()

    np.random.seed(98765)
    promt_dataset = pd.read_csv(f'../data_preprocessed/{args.data_name }/prompt_set_timestamped.csv')
    promt_dataset = promt_dataset[promt_dataset.rating > 3]
    for _, group in data_grouped_by_user:

        n_items_u = len(group)

        assert len(group.userId.unique()) == 1
        user_prompts_user = promt_dataset[promt_dataset.userId == group.userId.unique()[0  ]]
        group_train = group[group.itemId.isin(user_prompts_user.itemId)]
        group_val = group[~group.itemId.isin(user_prompts_user.itemId)]
        group_val = group_val[group_val.rating > 3]
        if len (group_val) == 0:
            print(group)

        tr_list.append(group_train)
        te_list.append(group_val)
        
        
    data_tr = pd.concat(tr_list)
    data_te = pd.concat(te_list)
    return data_tr, data_te



def numerize(tp, profile2id, show2id):
    
    uid = tp['userId'].apply(lambda x: profile2id[x] if x in profile2id else np.nan)
    sid = tp['itemId'].apply(lambda x: show2id[x] if x in show2id else np.nan)
    
    ratings = tp['rating']
    titles = tp['title']

    genres = tp['genres']
    out_df = pd.DataFrame(data={'uid': uid, 'sid': sid,'rating':ratings,'genres':genres,'title':titles}, columns=['uid', 'sid','rating','genres','title'])
    #filter out nans 
    out_df = out_df[out_df.uid.notna()]
    out_df = out_df[out_df.sid.notna()]


    #assert there are no more nan rows 
    assert out_df.isna().sum().sum() == 0
    return out_df 





In [4]:


print("Load and Preprocess Movielens-20m dataset")
args = parse_args()

# Load Data
DATA_DIR = f'../data_preprocessed/{args.data_name}/'
if args.data_name =='ml-1m':
# raw_data = pd.read_csv(os.path.join(DATA_DIR, 'ratings.csv'), header=0) READ LIKE THIS BUT IT IS A .DAT FILE WITH SEP = ::
    raw_data = pd.read_csv(f'../data/{args.data_name}/ratings.dat', sep="::", header=None, encoding='ISO-8859-1')
    raw_data.columns = ['userId', 'itemId', 'rating', 'timestamp']



elif args.data_name == 'books':
    raw_data = pd.read_csv(f'../data/{args.data_name}/ratings.csv', header=0)
    raw_data.rename(columns={'book_id':'itemId','review/time':'timestamp','Title':'title','review/score':'rating','User_id':'userId','categories':'genres'}, inplace=True)
    
#the lines below are a test that reduce the data
'''
raw_data = raw_data[raw_data['rating'] > 3]
print(f"{len(raw_data.userId.unique())=}")
print(f"{len(raw_data.itemId.unique())=}")
'''
raw_data_ratings  = raw_data[raw_data['rating'] > 3]

print(f"{len(raw_data.userId.unique())=}")
print(f"{len(raw_data.itemId.unique())=}")
# raw_data = raw_data[raw_data['rating'] > 3]


# Filter Data
raw_data_ratings, user_activity, item_popularity = filter_triplets(raw_data_ratings)
raw_data = raw_data[raw_data['userId'].isin(raw_data_ratings.userId.unique())]



# Filter Data
raw_data, user_activity, item_popularity = filter_triplets(raw_data)


# Shuffle User Indices
unique_uid = pd.Series(raw_data.userId.unique())

np.random.seed(98765)
idx_perm = np.random.permutation(unique_uid.size)

unique_uid = unique_uid[idx_perm]

n_users = unique_uid.size
print(f"{n_users=}")

n_heldout_users = 250

# Split Train/Validation/Test User Indices
train_data = pd.read_csv(f'../data_preprocessed/{args.data_name }/train_leave_one_out_.csv')

strong_generalization_set = pd.read_csv(f'../data_preprocessed/{args.data_name }/strong_generalization_set_.csv')
#filter such that the rating is higher than 3 
strong_generalization_set = strong_generalization_set[strong_generalization_set.rating > 3]
print(f"{len(pd.unique(train_data['itemId']))=}")

train_data = train_data[train_data.rating > 3]


print(f"{len(pd.unique(train_data['itemId']))=}")


with open(f'../saved_user_summary/{args.data_name }/user_summary_gpt4_.json','r') as f:
    prompts = json.load(f)
    prompts_keys = set([int(float(k)) for k,v in prompts.items()])

strong_gen_val_user_set = random.sample(list(strong_generalization_set.userId.unique()),int(len(strong_generalization_set.userId.unique())/2))
strong_generalization_set_val =strong_generalization_set[strong_generalization_set.userId.isin(strong_gen_val_user_set)]
strong_generalization_set_test = strong_generalization_set[~strong_generalization_set.userId.isin(strong_gen_val_user_set)]

unique_uid = unique_uid[unique_uid.isin(prompts_keys) ]

tr_users = unique_uid[unique_uid.isin(train_data.userId.unique())]
print(f"{len(tr_users)=}")

vd_users = unique_uid[unique_uid.isin(strong_generalization_set_val.userId.unique())]
print(f"{len(vd_users)=}")

te_users = unique_uid[unique_uid.isin(strong_generalization_set_test.userId.unique())]
print(f"{len(te_users)=}")

train_plays = raw_data.loc[raw_data['userId'].isin(tr_users)]

unique_sid = pd.unique(train_plays['itemId'])
print(f"{len(unique_sid)=}")

show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))
profile2id = dict((pid, i) for (i, pid) in enumerate(prompts_keys) )

prompts = {profile2id[int(float(k))]:v for k,v in prompts.items() }
#save the two dicts abovec to pickle 
import pickle

with open(os.path.join(DATA_DIR, 'show2id.pkl'), 'wb') as f:
    pickle.dump(show2id, f, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(DATA_DIR, 'profile2id.pkl'), 'wb') as f:
    pickle.dump(profile2id, f, pickle.HIGHEST_PROTOCOL)


if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

with open(os.path.join(DATA_DIR, 'unique_sid.txt'), 'w') as f:
    for sid in unique_sid:
        f.write('%s\n' % sid)

vad_plays = raw_data.loc[raw_data['userId'].isin(vd_users)]

# vad_plays = vad_plays.loc[vad_plays['itemId'].isin(unique_sid)]

vad_plays_tr, vad_plays_te = split_train_test_proportion(vad_plays)
#save vad_plays_tr to use as validation set for the training of the VAE

# vad_plays_tr.to_csv(os.path.join(DATA_DIR, 'validation_tr_.csv'), index=False)
# vad_plays_te.to_csv(os.path.join(DATA_DIR, 'validation_te_.csv'), index=False)
# #make sure the tr has the same users as the test 
# vad_plays_tr = vad_plays_tr.loc[vad_plays_tr['userId'].isin(vad_plays_te.userId)]

# test_plays = raw_data.loc[raw_data['userId'].isin(te_users)]
# test_plays = test_plays.loc[test_plays['itemId'].isin(unique_sid)]

# test_plays_tr, test_plays_te = split_train_test_proportion(test_plays)
# test_plays_tr = test_plays_tr.loc[test_plays_tr['userId'].isin(test_plays_te.userId)]


# train_data = numerize(train_plays, profile2id, show2id)
# #make sure the train_data is above 3 
# train_data = train_data[train_data.rating > 3]

# train_data.to_csv(os.path.join(DATA_DIR, 'train.csv'), index=False)

# vad_data_tr = numerize(vad_plays_tr, profile2id, show2id)

# vad_data_tr.to_csv(os.path.join(DATA_DIR, 'validation_tr.csv'), index=False)

# vad_data_te = numerize(vad_plays_te, profile2id, show2id)


# vad_data_te.to_csv(os.path.join(DATA_DIR, 'validation_te.csv'), index=False)

# test_data_tr = numerize(test_plays_tr, profile2id, show2id)
# test_data_tr.to_csv(os.path.join(DATA_DIR, 'test_tr.csv'), index=False)

# test_data_te = numerize(test_plays_te, profile2id, show2id)
# test_data_te.to_csv(os.path.join(DATA_DIR, 'test_te.csv'), index=False)

# print("Done!")

Load and Preprocess Movielens-20m dataset
len(raw_data.userId.unique())=3759
len(raw_data.itemId.unique())=15271
n_users=3759
len(pd.unique(train_data['itemId']))=14731
len(pd.unique(train_data['itemId']))=14137
len(tr_users)=3034
len(vd_users)=250
len(te_users)=250
len(unique_sid)=14871


In [5]:
vad_plays_tr.groupby('userId').count()

,Id,title,Price,profileName,review/helpfulness,rating,timestamp,review/summary,review/text,itemId,genres
userId,,,,,,,,,,,
2,43,43,3,43,43,43,43,43,43,43,43
8,30,30,6,30,30,30,30,30,30,30,30
13,19,19,4,19,19,19,19,19,19,19,19
26,41,41,7,41,41,41,41,41,41,41,41
30,26,26,5,26,26,26,26,26,26,26,26
...,...,...,...,...,...,...,...,...,...,...,...
3699,25,25,1,25,25,25,25,25,25,25,25
3715,12,12,0,12,12,12,12,12,12,12,12
3723,21,21,3,21,21,21,21,21,21,21,21


In [6]:
vad_plays_te.groupby('userId').count()

,Id,title,Price,profileName,review/helpfulness,rating,timestamp,review/summary,review/text,itemId,genres
userId,,,,,,,,,,,
2,6,6,0,6,6,6,6,6,6,6,6
8,35,35,2,35,35,35,35,35,35,35,35
13,2,2,0,2,2,2,2,2,2,2,2
26,224,224,21,224,224,224,224,224,224,224,224
30,2,2,0,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...
3699,2,2,1,2,2,2,2,2,2,2,2
3715,2,2,0,2,2,2,2,2,2,2,2
3723,2,2,0,2,2,2,2,2,2,2,2


In [7]:
#ge tthe users in the te set that are not in the tr set 
missing = set(vad_plays_tr.userId.unique()) - set(vad_plays_te.userId.unique())

In [8]:
missing

set()

In [9]:
data_grouped_by_user = vad_plays.groupby('userId')
tr_list, te_list = list(), list()
group = data_grouped_by_user.get_group(2729)
np.random.seed(98765)
promt_dataset = pd.read_csv(f'../data_preprocessed/{args.data_name }/prompt_set_timestamped.csv')
# promt_dataset = promt_dataset[promt_dataset.rating ]
user_prompts_user = promt_dataset[promt_dataset.userId == 2729]
group_train = group[group.itemId.isin(user_prompts_user.itemId)]
group_val = group[~group.itemId.isin(user_prompts_user.itemId)]
# group_val = group_val[group_val.rating > 3]


In [10]:
user_prompts_user.shape

(25, 12)

In [11]:
group_train.sort_values('timestamp',inplace=True)
group_train.shape

/tmp/ipykernel_3013700/2518034351.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group_train.sort_values('timestamp',inplace=True)


(25, 12)

In [12]:
#find the item id nmot in group but in prompt_datset for the user
group_train[group_train.itemId.isin(user_prompts_user.itemId)].shape


(25, 12)